In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as utils
import getData

In [2]:
senLen = 7
indexSize = 15144
embeddingSize = 300
kernelSizes = [3,4,5]
numFilters = 3
embeddings = None
XY = None
model = None
lossFn = nn.CrossEntropyLoss().type(torch.FloatTensor)
opt = None

In [ ]:
def makeModel(lr):
    model = nn.Sequential(
            nn.Conv2d(1,200,kernel_size=(3,embeddingSize),stride=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(200),
            nn.Dropout(p=0.2),
            '''nn.Conv2d(200,300,(4,1),stride=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(300),
            nn.Dropout(p=0.2),
            nn.Conv2d(300,500,(5,1),stride=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(500),
            nn.Dropout(p=0.2),'''
            nn.MaxPool2d((50,1)),
            Flatten(), 
            nn.Linear(500,2),

In [59]:
def makeEmbeddings(numWords,index,X,senWords,embed):
    inputData = torch.zeros([len(X),1,senWords,embeddingSize],dtype=torch.float64)
    for i,sen in enumerate(X):
        senIndex = torch.zeros([1,senWords,embeddingSize],dtype=torch.float64)
        for j,word in enumerate(sen):
            senIndex[0,j]= embed[j-1]
        inputData[i][0] = Variable(torch.DoubleTensor(senIndex[0]))
    return inputData

def makeModel(lr):
    model = nn.Sequential(
            nn.Conv2d(1,200,kernel_size=(3,embeddingSize),stride=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(200),
            nn.Dropout(p=0.2),
            Flatten(), 
            nn.Linear(1000,2),
    )
    opt = optim.Adam(model.parameters(),lr=lr)
    return model,opt

def train(numEpochs=1):
    print("Model",model)
    bestAcc = 0
    for epoch in range(numEpochs):
        print("Epoch #",epoch)
        total_loss = 0
        model.train()
        for i,(x,y) in enumerate(XY):
            xVar = Variable(x).float()
            yVar = Variable(y).type(torch.LongTensor)
            scores = model(xVar)
            loss = lossFn(scores,yVar)
            if i == 0: 
                print("i = %d, loss = %.4f" % (i + 1, loss.data[0]))
            if (i + 1) % 100 == 0:
                print("i = %d, loss = %.4f " % (i + 1, loss.data[0]))
            opt.zero_grad()
            loss.backward()
            total_loss += loss.data[0] 
            opt.step()
        currAcc = checkAccuracy(XY)
        if currAcc > bestAcc: 
            bestAcc = currAcc
            save_checkpoint(model,'model.pt')
        l = float(total_loss/float(i))
        print("total_loss %.4f"% l)
              
def checkAccuracy(dataSet):
    #if(self.XY.dataset.train):
    #    print('Checking accuracy on validation set')
    #else:
    #    print('Checking accuracy on test set')
    numCorrect = 0
    numSamples = 0
    model.eval()
    for i,(x,y) in enumerate(dataSet):
        xVar = Variable(x).float()
        yVar = Variable(y).type(torch.LongTensor)
        scores = model(xVar)
        _,preds = scores.data.cpu().max(1)
        numCorrect += (preds == yVar).sum()
        numSamples += preds.size(0)
    acc = float(numCorrect)/numSamples
    return acc
    print('Got %d / %d correct (%.2f)' % (numCorrect, numSamples, 100 * acc))

class Flatten(nn.Module):
    def forward(self,x):
        N,C,H,W = x.size()
        return x.view(N,-1)

In [5]:
print('calling getData...')
X,index,XY,Y = getData.runTrain()

calling getData...
med 7
15144
true
max 7
en 41646


In [6]:
Xte,XYte,Yte,max_sent_ = getData.getTest(index,max_sent_len)

SyntaxError: invalid syntax (<ipython-input-6-a826d48a6a09>, line 1)

In [7]:
print('making embeddings...')
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

making embeddings...


In [8]:
embeddings = torch.randn(len(index)+1,300)
for k,v in index.items():
    try: 
        embeddings[v-1].copy_(torch.from_numpy(word_vectors.get_vector(k)))
    except KeyError:
        continue 
embeddings[0] = 0

In [9]:
inputData = makeEmbeddings(len(index),index,X,senLen,embeddings)
#inputDataTe = makeEmbeddings(len(index),index,Xte,senLen,embeddings)

In [62]:
print('making model...')
model,opt=makeModel(lr=.01)
print(type(inputData), type(Y))
my_datasetTr = utils.TensorDataset(inputData, Y)
#my_datasetTe = utils.TensorDataset(inputDataTe, Yte)
my_dataloaderTr = utils.DataLoader(my_datasetTr,batch_size=64)
#my_dataloaderTe = utils.DataLoader(my_datasetTe,batch_size=64)
XY = my_dataloaderTr

making model...
<class 'torch.Tensor'> <class 'torch.Tensor'>


In [63]:
print('calling train...')
train(5)

calling train...
Model Sequential(
  (0): Conv2d(1, 200, kernel_size=(3, 300), stride=(1, 1))
  (1): ReLU(inplace)
  (2): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.2)
  (4): Flatten()
  (5): Linear(in_features=1000, out_features=2, bias=True)
)
Epoch # 0
i = 1, loss = 0.5772


/Users/user1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/user1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/user1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


i = 100, loss = 0.6612 
i = 200, loss = 0.4991 
i = 300, loss = 0.4459 
i = 400, loss = 0.6600 
i = 500, loss = 0.6822 
i = 600, loss = 0.6712 


/Users/user1/anaconda3/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Flatten. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


total_loss 0.6163
Epoch # 1
i = 1, loss = 1.9869
i = 100, loss = 0.6105 
i = 200, loss = 0.4933 
i = 300, loss = 0.4365 
i = 400, loss = 0.6395 
i = 500, loss = 0.6150 
i = 600, loss = 0.6637 
total_loss 0.6049
Epoch # 2
i = 1, loss = 1.1358
i = 100, loss = 0.6125 
i = 200, loss = 0.4898 
i = 300, loss = 0.4397 
i = 400, loss = 0.6327 
i = 500, loss = 0.6323 
i = 600, loss = 0.6707 
total_loss 0.5958
Epoch # 3
i = 1, loss = 1.4975
i = 100, loss = 0.6022 
i = 200, loss = 0.4936 
i = 300, loss = 0.4434 
i = 400, loss = 0.6482 
i = 500, loss = 0.6234 
i = 600, loss = 0.6672 
total_loss 0.5989
Epoch # 4
i = 1, loss = 1.5042
i = 100, loss = 0.6075 
i = 200, loss = 0.4866 
i = 300, loss = 0.4322 
i = 400, loss = 0.6308 
i = 500, loss = 0.6328 
i = 600, loss = 0.6633 
total_loss 0.5988


In [22]:
def save_checkpoint(state, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
save_checkpoint(model,  'model.pt')

/Users/user1/anaconda3/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Flatten. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [64]:
print('checking training accuracy...')
checkAccuracy(XY)

checking training accuracy...


0.4181433991259665

In [65]:


train(10)

Model Sequential(
  (0): Conv2d(1, 200, kernel_size=(3, 300), stride=(1, 1))
  (1): ReLU(inplace)
  (2): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.2)
  (4): Flatten()
  (5): Linear(in_features=1000, out_features=2, bias=True)
)
Epoch # 0
i = 1, loss = 1.5426


/Users/user1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/user1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/user1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


i = 100, loss = 0.6253 
i = 200, loss = 0.4849 
i = 300, loss = 0.4332 
i = 400, loss = 0.6334 
i = 500, loss = 0.6193 
i = 600, loss = 0.6602 


/Users/user1/anaconda3/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Flatten. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


total_loss 0.6006
Epoch # 1
i = 1, loss = 1.6935
i = 100, loss = 0.6091 
i = 200, loss = 0.4902 
i = 300, loss = 0.4318 
i = 400, loss = 0.6360 
i = 500, loss = 0.6155 
i = 600, loss = 0.6651 
total_loss 0.6011
Epoch # 2
i = 1, loss = 1.5827
i = 100, loss = 0.6157 
i = 200, loss = 0.4714 
i = 300, loss = 0.4359 
i = 400, loss = 0.6391 
i = 500, loss = 0.6238 
i = 600, loss = 0.6657 
total_loss 0.6012
Epoch # 3
i = 1, loss = 1.5415
i = 100, loss = 0.6185 
i = 200, loss = 0.4921 
i = 300, loss = 0.4487 
i = 400, loss = 0.6432 
i = 500, loss = 0.6730 
i = 600, loss = 0.6729 
total_loss 0.6027
Epoch # 4
i = 1, loss = 1.4742
i = 100, loss = 0.6134 
i = 200, loss = 0.4878 
i = 300, loss = 0.4304 
i = 400, loss = 0.6412 
i = 500, loss = 0.6287 
i = 600, loss = 0.6678 
total_loss 0.6013
Epoch # 5
i = 1, loss = 1.6975
i = 100, loss = 0.6032 
i = 200, loss = 0.4807 
i = 300, loss = 0.4341 
i = 400, loss = 0.6305 
i = 500, loss = 0.6236 
i = 600, loss = 0.6646 
total_loss 0.6031
Epoch # 6
i = 1, 

In [66]:
checkAccuracy(XY)

0.4181433991259665

In [15]:
print('checking validation accuracy...')
checkAccuracy(my_dataloaderTe)

checking validation accuracy...
Got 912 / 1821 correct (50.08)
